In [1]:
from tqdm import tqdm, trange
import json
from collections import namedtuple, defaultdict
import pandas as pd
import numpy as np
import torch
import os
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models.word2vec import Word2Vec
import random
from scipy.sparse import csr_matrix
import dgl

import pickle as pkl
import sys
sys.path.append('/mnt/nfs/zhangtl/utils')
from util import myout

Using backend: pytorch


## load

In [2]:
dataset = 'bus'
df = pd.read_csv(f'../../../01_process/data_relabel/{dataset}/Papers.csv') # 5353904
df.dropna(subset=['p_id', 'year'], inplace=True)
df.fillna({'refs': '', 'a_ids': '', 'v_id': '', 'fids': '', 'abst': ''}, inplace=True)
df

,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
0,"'Robbing Peter to Pay Paul"" Towards a Re-Conce...",2.999668e+09,1900.0,0.0,,3000273565,113497174.0,"67,5821,876,22824,37,15",
1,Changes in Federal and State Unemployment Insu...,2.915725e+09,1915.0,0.0,,2270227976,58891722.0,"67,265,61,525",
2,Consumer Expenditure Surveys Methods Symposium...,2.915972e+09,1915.0,0.0,,"2155669168,2805597960",58891722.0,"67,365,61,4656,15202",
3,"Job openings reach a new high in 2017, hires a...",2.893711e+09,1915.0,0.0,,"2699109867,2894114717",58891722.0,"61,67",
4,Measuring Labor Market Activity Today: Are the...,2.899681e+09,1915.0,1.0,,"2900346526,702506535",58891722.0,"464,9634,67,61",
...,...,...,...,...,...,...,...,...,...
951379,Understanding the differences between Chinese ...,3.037213e+09,2021.0,0.0,,"3037557308,3037533531,3037443334,2107505891",118637542.0,"15,67,59",
951380,Using Publicized Information to Determine the ...,3.037065e+09,2021.0,0.0,,"2764485510,2632733319,3037689016",83591844.0,"23,54,67,3041",
951381,Vocational Education and Training for Older Wo...,3.037220e+09,2021.0,0.0,,"2487327054,3036374174",138197279.0,"146,3,67,1346",
951382,Which are online shopping determinants Analyzi...,3.037917e+09,2021.0,0.0,,"3038037947,2751099055,3037979485,2345563520",161990933.0,"68290,2290,23,0,27",


In [3]:
start_year, end_year = 2000, 2022
drop_id = df[(df.year < start_year) | (df.year >= end_year)].index
print(f'Drop {len(drop_id)} rows') # 260112
papers = df.drop(drop_id) # 691271
papers

Drop 260111 rows


,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
260111,"""Globalization"": Modeling Technology Adoption ...",1.572765e+09,2000.0,79.0,"1489484807,1520398692,1720665485,1964473994,19...","240445305,2115074677,667469702",39307421.0,"143,810,67,30085",We study global adoption processes where the u...
260112,"""Honor"" Murders – Why the Perps Get off Easy",6.662752e+07,2000.0,18.0,,273429186,179627033.0,"98,9,7488,859",
260113,"""I am a Man!"": Race, Masculinity, and the 1968...",2.065042e+09,2000.0,10.0,"2003641387,2009140662,2039758851,2041933586,20...",2307979436,176330965.0,"306,8145,7185,67,7502",
260114,"""If only we'd known sooner"" developing knowled...",2.125439e+09,2000.0,23.0,"574672412,1495419098,1509637022,1520315996,156...",2481528218,154533451.0,"63,54,26,1380,15895,4054,1360,3920,515,1953",Managers of canceled innovation projects may b...
260115,"""It is what one does"": why people participate ...",2.003496e+09,2000.0,1518.0,"5414705,50261155,190890065,1518101824,15187321...","2095610511,2014929767",6144335.0,"2258,1280,15,1360,23,54,1282,5399,5409,26,199,...",Abstract Advances in information and communica...
...,...,...,...,...,...,...,...,...,...
951379,Understanding the differences between Chinese ...,3.037213e+09,2021.0,0.0,,"3037557308,3037533531,3037443334,2107505891",118637542.0,"15,67,59",
951380,Using Publicized Information to Determine the ...,3.037065e+09,2021.0,0.0,,"2764485510,2632733319,3037689016",83591844.0,"23,54,67,3041",
951381,Vocational Education and Training for Older Wo...,3.037220e+09,2021.0,0.0,,"2487327054,3036374174",138197279.0,"146,3,67,1346",
951382,Which are online shopping determinants Analyzi...,3.037917e+09,2021.0,0.0,,"3038037947,2751099055,3037979485,2345563520",161990933.0,"68290,2290,23,0,27",


## build graph

In [5]:
dataset = 'bus'
p_id2emb = pkl.load(open(f'../../../01_process/data_papers/{dataset}/p_id2emb.pkl', 'rb'))
myout(p_id2emb)
def update_idx(idx, dic, cnt, feats, rel, feat_dim, no_emb):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        if rel == 0:
            try:
                feats.append(torch.from_numpy(p_id2emb[idx]).to(torch.float32))
            except:
                feats.append(torch.rand(feat_dim).to(torch.float32)-1)
                no_emb += 1
        elif rel == 1:
            feats.append(torch.rand(feat_dim).to(torch.float32)+1)
        elif rel == 2:
            feats.append(torch.rand(feat_dim).to(torch.float32)+2)
        elif rel == 3:
            feats.append(torch.rand(feat_dim).to(torch.float32))
    return dic, cnt, feats, no_emb

p_id2emb : len=951383, dict([2999668136: [-0.38435513 -0.02309773  0.01890826  0.09264099 -0.06540252  0.25426486
  0.28358728 -0.27884638 -0.01874764 -0.02312707  0.01723664  0.23914391
 -0.07579181  0.32206938 -0.01650842 -0.19485569 -0.23048532  0.12914363
  0.09322546 -0.07421232  0.6637163   0.01646406 -0.09308308 -0.11911651
  0.1683344  -0.08467826 -0.06008676  0.01956851 -0.03913155 -0.088841
 -0.17371932  0.09311762 -0.05136457 -0.04628504  0.07411629  0.10916147
  0.10567741  0.10959639  0.24649368 -0.17431794  0.12163395  0.12411345
 -0.07620661 -0.05405273  0.06477507  0.3359203   0.20388564  0.00133745
 -0.13752297  0.09026892  0.00920943  0.23066929  0.17430007  0.06153947
  0.47288132 -0.33030465  0.38849714  0.04654486  0.3409918  -0.11961805
  0.07205509 -0.27441397  0.15869498 -0.08819257  0.05640403  0.06739137
 -0.16322209  0.06658246 -0.34838906 -0.01579924  0.19766213 -0.17577218
 -0.02606395 -0.2333377  -0.04564463 -0.04954341 -0.18728772  0.0202555
 -0.04274923 

In [6]:
start_year, end_year = 2000, 2022
feat_dim = 128

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
for i in trange(len(papers)):
    ts = int(papers['year'].iloc[i])
    p_id = int(papers['p_id'].iloc[i])
    refs = papers['refs'].iloc[i]
    
    id2nid, cnt, feats, no_emb = update_idx(p_id, id2nid, cnt, feats, 0, feat_dim, no_emb)
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            if ref != '':
                ref = int(ref)
                id2nid, cnt, feats, no_emb = update_idx(ref, id2nid, cnt, feats, 0, feat_dim, no_emb)
                lst.append((id2nid[p_id], id2nid[ref], 0, ts))
        
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])
myout(feat, src, tgt, rel, tsp)

100%|██████████| 691272/691272 [01:57<00:00, 5858.73it/s]


feat : shape=torch.Size([3928209, 128])
tensor([[-0.1871,  0.0140,  0.1467,  ..., -0.0847, -0.3531,  0.1211],
        [-0.7970, -0.5914, -0.2980,  ..., -0.6728, -0.1607, -0.3570],
        [-0.2957,  0.0552,  0.1578,  ..., -0.1217, -0.3247,  0.1083],
        ...,
        [-0.2394, -0.1490,  0.0262,  ..., -0.1757, -0.3574,  0.1743],
        [-0.1929, -0.0494,  0.1787,  ..., -0.0965, -0.3065,  0.0801],
        [-0.1969, -0.2608, -0.6842,  ..., -0.8212, -0.0254, -0.2728]])
src : shape=torch.Size([19955649]), tensor([      0,       0,       0,  ..., 3928207, 3928207, 3928207])
tgt : shape=torch.Size([19955649]), tensor([      1,       2,       3,  ..., 3586889, 3068962, 3928208])
rel : shape=torch.Size([19955649]), tensor([0, 0, 0,  ..., 0, 0, 0])
tsp : shape=torch.Size([19955649]), tensor([2000, 2000, 2000,  ..., 2021, 2021, 2021])


In [7]:
graph = dgl.graph((src, tgt), num_nodes=len(feat))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_nid'] = torch.arange(len(feat))

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

Graph(num_nodes=3928209, num_edges=19955649,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [8]:
dgl.save_graphs(f'../data/{dataset}/graph.bin', [graph])

In [9]:
json.dump(id2nid, open(f'../data/{dataset}/id2nid.json', 'w'))

## gen cites

In [10]:
cites = {}
print(start_year, end_year)
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

papers.fillna({'refs': ''}, inplace=True)
for i in trange(len(papers)):
    year = int(papers['year'].iloc[i])
    refs = papers['refs'].iloc[i]
    if len(refs)>0:
        rlst = refs.split(',')
        rlst = [int(item) for item in rlst]
        for ref in rlst:
            if ref != '':
                cites[year][ref] += 1

2000 2022


100%|██████████| 691272/691272 [00:21<00:00, 32076.71it/s]


In [11]:
tsp = graph.edata['ts']
ts_vals, ts_cuts = np.unique(tsp.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(tsp.numpy())]

num_ts = len(ts_vals)
ts_infos = np.stack([ts_vals, ts_cuts[0:num_ts], ts_cuts[1:num_ts+1]]).transpose()
myout(ts_cuts, ts_vals, ts_infos)

ts_cuts : len=23, list([0, 244220, 504248, ..., 18395726, 19954635, 19955649])
ts_vals : shape=(22,), [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021]
ts_infos : shape=(22, 3)
[[    2000        0   244220]
 [    2001   244220   504248]
 [    2002   504248   804985]
 [    2003   804985  1149458]
 [    2004  1149458  1542737]
 [    2005  1542737  1999737]
 [    2006  1999737  2552332]
 [    2007  2552332  3181546]
 [    2008  3181546  3899279]
 [    2009  3899279  4716490]
 [    2010  4716490  5613683]
 [    2011  5613683  6617103]
 [    2012  6617103  7700613]
 [    2013  7700613  8899557]
 [    2014  8899557 10187760]
 [    2015 10187760 11575276]
 [    2016 11575276 13046580]
 [    2017 13046580 14727341]
 [    2018 14727341 16300065]
 [    2019 16300065 18395726]
 [    2020 18395726 19954635]
 [    2021 19954635 19955649]]


In [13]:
labels = {}
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = ts_infos[np.where(ts_infos[:, 0]==year)[0][0], 1:]
    nids = graph.edges()[0][left:right].unique().tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf

2020: 100%|██████████| 1/1 [00:00<00:00, 155.04it/s, pdf=30976, year=2020]
2021: 0it [00:00, ?it/s]


In [14]:
labels[2005]

,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,2106707824,5201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2043332151,15059,4.0,10.0,5.0,8.0,4.0,9.0,17.0,17.0,17.0,16.0,21.0,19.0,14.0,12.0,6.0,0.0
2,2120351738,19374,2.0,4.0,1.0,1.0,3.0,2.0,4.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,1.0,0.0
3,2089949321,21964,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2097685531,24915,2.0,4.0,7.0,5.0,9.0,15.0,8.0,7.0,15.0,9.0,10.0,16.0,8.0,8.0,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15942,2012953831,676226,1.0,2.0,1.0,3.0,1.0,3.0,0.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
15943,1992094292,676235,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15944,2049329352,676236,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
15945,1986510715,676241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
pkl.dump(labels, open(f'../data/{dataset}/labels.pkl', 'wb'))

## cum log labels

In [16]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-2): # 2000, 2020
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[2005]

21


,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,2106707824,5201,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2043332151,15059,1.609438,2.708050,2.995732,3.332205,3.465736,3.713572,4.060443,4.317488,4.521789,4.682131,4.859812,4.997212,5.087596,5.159055,5.192957,5.192957
2,2120351738,19374,1.098612,1.945910,2.079442,2.197225,2.484907,2.639057,2.890372,2.944439,3.091043,3.178054,3.258096,3.332205,3.433987,3.496508,3.526361,3.526361
3,2089949321,21964,0.693147,0.693147,0.693147,0.693147,0.693147,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.609438,1.609438,1.609438,1.609438
4,2097685531,24915,1.098612,1.945910,2.639057,2.944439,3.332205,3.761200,3.931826,4.060443,4.290460,4.406719,4.521789,4.682131,4.753590,4.820282,4.882802,4.882802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15942,2012953831,676226,0.693147,1.386294,1.609438,2.079442,2.197225,2.484907,2.484907,2.639057,2.772589,2.890372,2.890372,2.944439,2.944439,2.944439,2.944439,2.944439
15943,1992094292,676235,0.000000,0.693147,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612
15944,2049329352,676236,0.000000,0.000000,1.098612,1.098612,1.098612,1.386294,1.386294,1.386294,1.386294,1.609438,1.791759,1.945910,1.945910,1.945910,1.945910,1.945910
15945,1986510715,676241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
pkl.dump(labels_cum_log, open(f'../data/{dataset}/labels_cum_log.pkl', 'wb'))